# Data Preprocessor for the RNN
The RNN takes z, the encoding of a certain image, and predicts z', the encoding of the image following. So, to make data to train the RNN on, we need to make pairs of z and z' from our pictoral data.

## Loading stuff into the Kernel

Import Dependencies:

In [ ]:
# imports
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import save_model
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import joblib
# added so that cv2 gets installed in kernal
# import sys
# !{sys.executable} -m pip install opencv-python
# commented the above code, it started working, idk why
# if code not working try uncommenting the above
import cv2
import random
import glob

# Data (preprocessed from Data Processing Script)

## (And Reshape `x_train`)

In [ ]:


train_data = joblib.load("images/train_data.z")
print(train_data.shape[2])


img_width  = train_data.shape[1]
img_height = train_data.shape[2]
num_channels = 1
x_train = train_data.reshape(train_data.shape[0], img_height, img_width, num_channels)

input_shape = (img_height, img_width, num_channels)
print(input_shape)

# Encoder  

load the vae (have to make the architecture again, make sure the code below matches the code in the Data Prepper/VAE Trainer)

## `sample_z()` Function:

### REPARAMETERIZATION TRICK

Define sampling function to sample from the distribution  Reparameterize sample based on the process defined by Gunderson and Huang into the shape of: mu + sigma squared x eps 

This is to allow gradient descent to allow for gradient estimation accurately.

### `z`:  
sample vector from the latent distribution 
z is the labda custom layer we are adding for gradient descent calculations using mu and variance (sigma)

In [ ]:
latent_dim = 2048

input_img = Input(shape=input_shape, name='encoder_input')
x = Conv2D(64, 3, padding='same', activation='relu')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Dropout(0.2)(x)
x = Conv2D(128, 3, padding='same', activation='relu')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Dropout(0.2)(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Dropout(0.2)(x)
x = Conv2D(32, 3, padding='same', activation='relu')(x)

conv_shape = K.int_shape(x)
x = Flatten()(x)
x = Dense(latent_dim*2, activation='relu')(x)

z_mu = Dense(latent_dim, name='latent_mu')(x)
z_sigma = Dense(latent_dim, name='latent_sigma')(x)

def sample_z(args):
    z_mu, z_sigma = args
    eps = K.random_normal(shape=(K.shape(z_mu)[0], K.int_shape(z_mu)[1]))
    return z_mu + K.exp(z_sigma / 2) * eps

z = Lambda(sample_z, output_shape=(latent_dim, ), name='z')([z_mu, z_sigma])

encoder = Model(input_img, [z_mu, z_sigma, z], name='encoder')
print(encoder.summary())


# Decoder 

`decoder_input` : decoder takes the latent vector as input

`x = Dense()`: Need to start with a shape that can be remapped to original image shape as we want our final utput to be same shape original input. So, add dense layer with dimensions that can be reshaped to desired output shape

In [ ]:
x = Dense(conv_shape[1]*conv_shape[2]*conv_shape[3], activation='relu')(decoder_input)
# reshape to the shape of last conv. layer in the encoder, so we can 
x = Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(x)
# upscale (conv2D transpose) back to original shape
# use Conv2DTranspose to reverse the conv layers defined in the encoder
x = Conv2DTranspose(32, 3, padding='same', activation='relu',strides=(2, 2))(x)
x = Conv2DTranspose(32, 3, padding='same', activation='relu')(x)
x = Conv2DTranspose(64, 3, padding='same', activation='relu',strides=(2, 2))(x)
x = Conv2DTranspose(64, 3, padding='same', activation='relu')(x)
x = Conv2DTranspose(64, 3, padding='same', activation='relu',strides=(2, 2))(x)
#Can add more conv2DTranspose layers, if desired. 
#Using sigmoid activation
x = Conv2DTranspose(num_channels, 3, padding='same', activation='sigmoid', name='decoder_output')(x)

# Define and summarize decoder model
decoder = Model(decoder_input, x, name='decoder')
decoder.summary()

# apply the decoder to the latent sample 
z_decoded = decoder(z)

# Loss Function

## `y`: 
- apply the custom loss to the input images and the decoded latent distribution sample
- y is basically the original image after encoding input img to mu, sigma, z and decoding sampled z values. This will be used as output for vae

In [ ]:
class CustomLayer(keras.layers.Layer):
    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        
        # Reconstruction loss (as we used sigmoid activation we can use binarycrossentropy)
        recon_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        
        # KL divergence
        kl_loss = -5e-4 * K.mean(1 + z_sigma - K.square(z_mu) - K.exp(z_sigma), axis=-1)
        return K.mean(recon_loss + kl_loss)

    # add custom loss to the class
    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x


y = CustomLayer()([input_img, z_decoded])

# Load Encoder

In [ ]:
vae = Model(input_img, y, name = 'vae')
vae.load_weights(os.getcwd() + "\\models\\vae.h5")
encoder = Model(vae.input, vae.layers[15].output)

## Data Preprocessor

In [ ]:
# preprocesses data as before, but puts markers inbetween to seperate the data

# TODO: might be times between photos within the folders, so maybe will need to look closer at where to put delimiters

os.chdir("images")

# save boolean of if data has been saved already or not so can negate future
#    cells to avoid the code breaking
data_exists = os.path.exists("train_data_rnn.z")
# constant for sizing
IMG_SIZE = 128

if data_exists:
    print("train_data_rnn.z already exists, if this notebook is run to completion the old data will be replaced.")
    
data = []
path = os.getcwd()
print(path)

def create_data():   
    count = 0
    for folder in os.listdir(path):
        if "train_data" in folder:  # skips any train data files, as that should be the only non-directory item in images
            continue
        print("FOLDER: ",folder)
        # added + "/" + to below to make it work
        for filename in os.listdir(path + "\\" + folder):
            # changed to NEF (That's what I have the images saved as, may need to change back to JPG in future)
            if(".NEF" in filename):
                # added slash here too
                temp_path = path + "/" + folder + "/" + filename
                count += 1
                try:
                    img_array = cv2.imread(temp_path)
                    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
                    img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    data.append(img_array)
                    print("image processed..." + str(count) , end="\r")

                except Exception as e:
                    pass
        data.append("|")

create_data()

In [ ]:
# reshape data
print(train_data.shape[2])

# Reshape 
img_width  = train_data.shape[1]
img_height = train_data.shape[2]
num_channels = 1
x_train = train_data.reshape(train_data.shape[0], img_height, img_width, num_channels)


input_shape = (img_height, img_width, num_channels)
print(input_shape)

In [ ]:
# goes through the seperated data and pairs it
paired_data = []

for i in range(len(x_train)-1):
    if x_train[i] == "|" or x_train[i+1] == "|":
        continue
    else:
        paired_data.append([x_train[i], x_train[i+1]])

In [ ]:
print(paired_data[0][0].shape)

In [ ]:
# runs the paired data through the encoder to get the latent vectors
z_vals = []

for pair in paired_data:
    input1 = pair[0][None,:,:,:]
    input2 = pair[1][None,:,:,:]
    z1 = encoder.predict(input1)
    z2 = encoder.predict(input2)
    z_vals.append([z1, z2])

In [ ]:
# finished, now save the data
joblib.dump(z_vals, "train_data_rnn.z")

# Model Trainer
(As opposed to the controller part of the model)

## Loading stuff into the kernel:
Imports and training data (don't need the encoder, train data is already converted)

In [ ]:
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape, MaxPooling2D,LSTM, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import save_model
import numpy as np
import matplotlib.pyplot as plt
import os
import joblib
import cv2
import random
import glob

In [ ]:
data = joblib.load("images/train_data_rnn.z")

In [ ]:
train_data = np.array([np.array(p[0])for p in data])
answers = np.array([np.array(p[1]) for p in data])
print(train_data[0])
print(answers[0])
print(np.shape(train_data[0]))
z_len = np.shape(train_data[0])[-1]
print(z_len)

# Building the RNN

In [ ]:
input_to_rnn = Input(shape=(1,z_len))

x = LSTM(z_len, return_sequences=True)(input_to_rnn)
x = Dropout(0.2)(x)
x = Dense(z_len))(x)
x = Dropout(0.2)(x)

In [ ]:
rnn = Model(input_to_rnn, output, name='rnn')

In [ ]:
rnn.summary()

In [ ]:
rnn.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
print(np.shape(answers[0]))

# Training

after testing with z length = 2048, 10 epochs seems like the best. val_loss goes from 0.90 to 0.82, but goes back to back to 0.83 for overfitting. If more data added, could potentially do more epochs also, doesn't take that long to run due to small dimensionality of inputs and outputs

In [ ]:
rnn.fit(train_data, answers, epochs=10, verbose=1, batch_size=32, validation_split=0.2)


In [ ]:
rnn.save_weights('models/rnn.h5')